# Setup of methanol in (specified solvent)

Here we are setting up solvation free energy calculations for a single molecule of methanol in (you specify the solvent). We will set up the system (with GAFF parameters) using SolvationToolkit in AMBER and GROMACS formats for use in GROMACS and Yank for solvation free energy estimation beginning from those formats. We will then also generate OpenMM Systems using ParmEd and serialize to XML so we can test beginning Yank calculations from that format as well. Further, we will take the same system, assign SMIRNOFF99Frosst parameters, and dump to GROMACS and serialized OpenMM XML formats for use in GROMACS and Yank. The intention is to ensure we can reproduce the same solvation free energies in all cases.

## Do imports

In [10]:
from solvationtoolkit.solvated_mixtures import *
import parmed
from openforcefield.utils import get_data_filename, extractPositionsFromOEMol, generateTopologyFromOEMol
from openforcefield.typing.engines.smirnoff.forcefield_utils import *
from simtk.openmm.app import *
from simtk.openmm import XmlSerializer
from simtk import unit
from openforcefield.typing.engines.smirnoff import ForceField

## Prep system with SolvationToolkit

In [4]:
infinite_dilution = MixtureSystem()
infinite_dilution.addComponent(name='methanol', number=1)
#Specify your solvent here
solvent_name = 'cyclohexane' #Needs to be a name which is compatible with file names
infinite_dilution.addComponent(name=solvent_name, number=300)
infinite_dilution.build(amber=True, gromacs=True)

//anaconda/lib/python3.5/site-packages/pandas/io/parsers.py:1961: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
//anaconda/lib/python3.5/site-packages/pandas/io/parsers.py:1963: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())



# Mixture 

tolerance 2.000000
filetype pdb
output /var/folders/k0/v23_69_j415062nr1f1y8sr00000gn/T/tmpnju1k6hs.pdb
add_amber_ter


structure /var/folders/k0/v23_69_j415062nr1f1y8sr00000gn/T/tmpt1_pbokx.pdb
  number 1 
  inside box 0. 0. 0. 38.229686 38.229686 38.229686
end structure

structure /var/folders/k0/v23_69_j415062nr1f1y8sr00000gn/T/tmpu4a3g4eg.pdb
  number 300 
  inside box 0. 0. 0. 38.229686 38.229686 38.229686
end structure


# Mixture 

tolerance 2.000000
filetype pdb
output /var/folders/k0/v23_69_j415062nr1f1y8sr00000gn/T/tmpnju1k6hs.pdb
add_amber_ter


structure /var/folders/k0/v23_69_j415062nr1f1y8sr00000gn/T/tmpt1_pbokx.pdb
  number 1 
  inside box 0. 0. 0. 38.229686 38.229686 38.229686
end structure

structure /var/folders/k0/v23_69_j415062nr1f1y8sr00000gn/T/tmpu4a3g4eg.pdb
  number 300 
  inside box 0. 0. 0. 38.229686 38.229686 38.229686
end structure


source leaprc.gaff
source oldff/leaprc.ff99SB
ZTZ = loadmol2 in0.mol2
ZYT = loadmol2 in1.mol2
box = loadPdb tbox.

## Use ParmEd to make GROMACS files into OpenMM system then serialize to XML for use in Yank

In [6]:
top = parmed.load_file('data/gromacs/methanol_%s_1_300.top' % solvent_name)
gro = parmed.load_file('data/gromacs/methanol_%s_1_300.gro' % solvent_name)
top.box = gro.box
system = top.createSystem(nonbondedMethod = app.PME, nonbondedCutoff=1.0*unit.nanometer)

# Make dir for serialized systems if needed
import os
if not os.path.isdir( 'data/openmm'): os.mkdir('data/openmm')

# Save
serialized = XmlSerializer.serialize(system)
outfile = open('data/openmm/methanol_%s_gromacs_gaff.xml' % solvent_name, 'w')
outfile.write(serialized)
outfile.close()

## Assign SMIRNOFF parameters to system

In [14]:
# Load forcefield
ff = ForceField( get_data_filename('forcefield/smirnoff99Frosst.ffxml'))
from openforcefield.typing.engines.smirnoff import PME

# Load oemols
from openeye.oechem import *
mols = []
flavor = oechem.OEIFlavor_Generic_Default | oechem.OEIFlavor_MOL2_Default | oechem.OEIFlavor_MOL2_Forcefield
for molname in [solvent_name, 'methanol']:
    ifs = oechem.oemolistream( os.path.join('data/monomers', '%s.mol2' % molname))
    ifs.SetFlavor( oechem.OEFormat_MOL2, flavor)
    mol = oechem.OEGraphMol()
    while oechem.OEReadMolecule(ifs, mol):
        oechem.OETriposAtomNames(mol)
        mols.append(oechem.OEGraphMol(mol))
        
# Load PDBFile
pdbfile = PDBFile( 'data/packmol_boxes/methanol_%s_1_300.pdb' % solvent_name)
system = ff.createSystem(pdbfile.topology, mols, nonbondedMethod=PME, nonbondedCutoff=1.0*unit.nanometer)

## Dump SMIRNOFF system to XML

In [15]:
serialized = XmlSerializer.serialize(system)
outfile = open('data/openmm/methanol_%s_smirnoff.xml' % solvent_name, 'w')
outfile.write(serialized)
outfile.close()

## Convert SMIRNOFF system to GROMACS and dump

In [16]:
structure = parmed.openmm.topsystem.load_topology(pdbfile.topology, system, pdbfile.positions)
structure.save( 'data/gromacs/methanol_%s_smirnoff.gro' % solvent_name, overwrite=True)
structure.save( 'data/gromacs/methanol_%s_smirnoff.top' % solvent_name, overwrite=True)

# Make a system with methanol in the gas phase and save
## Do original GAFF

In [17]:
# Add code here
# I had to construct a GROMACS topology/coordinate file for the monomer by hand as I don't have one generated
# I did it by copying the ones for the solvated system and removing the solvent
top = parmed.load_file('data/gromacs/methanol.top')
gro = parmed.load_file('data/gromacs/methanol.gro')
top.box = gro.box
system = top.createSystem(nonbondedMethod = app.NoCutoff)

# Make dir for serialized systems if needed
import os
if not os.path.isdir( 'data/openmm'): os.mkdir('data/openmm')

# Save
serialized = XmlSerializer.serialize(system)
outfile = open('data/openmm/methanol_gromacs_gaff.xml', 'w')
outfile.write(serialized)
outfile.close()

OSError: data/gromacs/methanol.top does not exist

## Do SMIRNOFF in XML

In [9]:
# Make Topology for methanol
topology = generateTopologyFromOEMol(mols[1])
system = ff.createSystem(topology, mols)

# Save
serialized = XmlSerializer.serialize(system)
outfile = open('data/openmm/methanol_smirnoff.xml', 'w')
outfile.write(serialized)
outfile.close()

## Do SMIRNOFF in GROMACS

In [10]:
structure = parmed.openmm.topsystem.load_topology(topology, system, gro.positions)
structure.save( 'data/gromacs/methanol_smirnoff.gro', overwrite=True)
structure.save( 'data/gromacs/methanol_smirnoff.top', overwrite=True)